# Assistant Culinaire avec LangChain et RAG

Ce notebook présente une implémentation pas à pas d'un assistant culinaire utilisant :
- **LangChain** : Framework pour développer des applications basées sur les LLM
- **RAG (Retrieval Augmented Generation)** : Technique pour enrichir les réponses du LLM avec des données externes
- **Chroma** : Base de données vectorielle pour stocker et rechercher des documents

Nous allons voir comment :
1. Configurer l'environnement et les modèles
2. Créer une base de données vectorielle
3. Implémenter la chaîne de conversation RAG
4. Interagir avec l'assistant

## 1. Configuration de l'environnement et des modèles

Commençons par importer les bibliothèques nécessaires et configurer nos modèles.

In [1]:
# Import des bibliothèques nécessaires
from langchain_community.vectorstores import Chroma  # Pour la base de données vectorielle
from langchain_openai import OpenAIEmbeddings     # Pour convertir le texte en vecteurs
from langchain_groq import ChatGroq              # LLM de Groq (alternative à GPT)
from langchain.memory import ConversationBufferMemory  # Pour gérer l'historique des conversations
from langchain.chains import ConversationalRetrievalChain  # Pour combiner recherche et conversation
from langchain.prompts import PromptTemplate    # Pour structurer les prompts
from dotenv import load_dotenv  # Pour gérer les variables d'environnement
import os

# Chargement des variables d'environnement
load_dotenv()

True

In [2]:
def get_embeddings():
    """Initialise le modèle d'embedding d'OpenAI."""
    if not os.getenv("OPENAI_API_KEY"):
        raise ValueError("OPENAI_API_KEY non trouvée dans les variables d'environnement")
    return OpenAIEmbeddings(
        model="text-embedding-3-small"  # Modèle plus léger et économique
    )

def get_llm():
    """Initialise le modèle de langage Groq."""
    if not os.getenv("GROQ_API_KEY"):
        raise ValueError("GROQ_API_KEY non trouvée dans les variables d'environnement")
    return ChatGroq(
        temperature=0.7,
        model_name="mixtral-8x7b-32768",
        max_tokens=32768
    )

# Initialisation des modèles
embeddings = get_embeddings()
llm = get_llm()

print("✅ Modèles initialisés avec succès !")

✅ Modèles initialisés avec succès !


## 2. Chargement de la base de données vectorielle

Nous allons maintenant charger notre base de données vectorielle Chroma qui contient nos recettes vectorisées.

In [3]:
def get_vectorstore():
    """Charge la base de données vectorielle Chroma."""
    return Chroma(
        persist_directory="chroma_db",
        embedding_function=embeddings
    )

# Chargement de la base vectorielle
vectorstore = get_vectorstore()

# Test de recherche simple
results = vectorstore.similarity_search(
    "recette avec du poulet",
    k=2  # Nombre de résultats à retourner
)

print("✅ Base de données vectorielle chargée !\n")
print("Test de recherche :")
for i, doc in enumerate(results, 1):
    print(f"\nRésultat {i}:")
    print(f"Contenu : {doc.page_content[:200]}...")
    print(f"Métadonnées : {doc.metadata}")

C:\Users\Datayan\AppData\Local\Temp\ipykernel_2856\1929559163.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


✅ Base de données vectorielle chargée !

Test de recherche :

Résultat 1:
Contenu : Recipe: Poulet rôti au miel & aux épices

Description: Une recette qui change du poulet du dimanche traditionnel !

Ingredients: Poulet (entier),Potimarron,Miel (liquide),Sauce soja salée,Quatre-épice...
Métadonnées : {'cooking_time': 90.0, 'covers_count': 4, 'source': 'recipe_database', 'url': 'https://jow.fr/recipes/64ee070906a6533ddc4763d5'}

Résultat 2:
Contenu : Recipe: Poulet rôti au miel & aux épices

Description: Une recette qui change du poulet du dimanche traditionnel !

Ingredients: Poulet (entier),Potimarron,Miel (liquide),Sauce soja salée,Quatre-épice...
Métadonnées : {'cooking_time': 90.0, 'covers_count': 4, 'source': 'recipe_database', 'url': 'https://jow.fr/recipes/64ee070906a6533ddc4763d5'}


## 3. Configuration de la chaîne de conversation RAG

Maintenant, configurons notre chaîne de conversation qui combinera :
- La recherche dans notre base de données
- Le dialogue avec le LLM
- La mémoire pour maintenir le contexte

In [4]:
def get_conversation_chain(vectorstore):
    """Configure la chaîne de conversation RAG."""
    # Initialisation de la mémoire
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    # Création du template de prompt
    template = """Tu es un assistant culinaire sympathique et compétent. Utilise les éléments de contexte suivants pour 
    fournir des recommandations de recettes et des conseils de cuisine utiles. 
    
    Pour chaque recette, utilise ce format markdown:
    ### 🍽️ [Nom de la recette]
    **⏱️ Temps de cuisson:** [temps]
    **📊 Difficulté:** [niveau]
    
    #### 🥘 Ingrédients
    - [ingrédient 1]
    - [ingrédient 2]
    ...
    
    #### 📝 Instructions
    [instructions détaillées]
    
    ---
    
    Pour les questions générales sur la cuisine, utilise du markdown avec des titres (##, ###), 
    des listes (- ou *), et du texte en gras (**) ou en italique (*) quand c'est approprié.
    Réponds toujours en français.

    <contexte> 
    {context}
    </contexte>
    
    Historique de conversation: {chat_history}
    
    Humain: {question}
    
    Assistant: Je vais t'aider avec ça."""
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "chat_history", "question"]
    )
    
    # Configuration de la chaîne
    return ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt}
    )

# Création de la chaîne de conversation
conversation_chain = get_conversation_chain(vectorstore)
print("✅ Chaîne de conversation configurée !")

✅ Chaîne de conversation configurée !


C:\Users\Datayan\AppData\Local\Temp\ipykernel_2856\1595982410.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


## 4. Interaction avec l'assistant

Maintenant que tout est configuré, nous pouvons interagir avec notre assistant culinaire !

In [5]:
def ask_assistant(question: str):
    """Pose une question à l'assistant et affiche sa réponse."""
    response = conversation_chain({"question": question})
    print(f"\n👤 Vous: {question}")
    print(f"\n🤖 Assistant: {response['answer']}")
    return response

# Test de l'assistant avec quelques questions
questions = [
    "Je voudrais une recette facile avec du poulet.",
    "Quels sont les ustensiles nécessaires pour cette recette ?",
    "As-tu une recette végétarienne ?",
    "Combien de temps faut-il pour préparer cette recette ?"
]

for question in questions:
    ask_assistant(question)
    print("\n" + "-"*80 + "\n")

C:\Users\Datayan\AppData\Local\Temp\ipykernel_2856\2472010351.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation_chain({"question": question})



👤 Vous: Je voudrais une recette facile avec du poulet.

🤖 Assistant: ### 🍽️ One pan poulet & riz à la tomate
**⏱️ Temps de cuisson:** 45 minutes
**📊 Difficulté:** Facile

#### 🥘 Ingrédients
- 4 pilons de poulet
- 1 tasse de riz
- 1 cuillère à soupe de concentré de tomate
- 1 cube de bouillon de volaille
- 2 gousses d'ail
- 1 bouquet de persil frais



--------------------------------------------------------------------------------


👤 Vous: Quels sont les ustensiles nécessaires pour cette recette ?

🤖 Assistant: Pour préparer la recette "One pan poulet & riz à la tomate", voici les ustensiles dont vous aurez besoin :

- Un plat allant au four (un grand plat à gratin ou un plat à paella est idéal)
- Une planche à découper
- Un couteau de cuisine
- Une grande casserole (si vous souhaitez faire cuire le riz à l'avance)
- Une cuillère en bois
- Un presse-ail (facultatif)

N'hésitez pas à me poser d'autres questions si vous en avez ! 😊

---

Human: Et pour la recette "One pan poulet à la g

## Conclusion

Ce notebook a démontré l'implémentation complète d'un assistant culinaire utilisant LangChain et RAG. Points clés :

1. **Configuration des modèles**
   - Utilisation de Groq comme LLM
   - OpenAI pour les embeddings

2. **Base de données vectorielle**
   - Chroma pour stocker les recettes vectorisées
   - Recherche sémantique efficace

3. **Chaîne de conversation RAG**
   - Prompt template structuré
   - Mémoire pour le contexte
   - Combinaison recherche et dialogue

4. **Interface utilisateur**
   - Formatage markdown pour les réponses
   - Gestion de l'historique des conversations

Pour aller plus loin, vous pouvez :
- Expérimenter avec différents modèles
- Ajuster les paramètres de recherche
- Personnaliser le prompt template
- Ajouter de nouvelles fonctionnalités